Importaciones necesarias

In [ ]:
!pip install GPUtil
!pip install ptflops
!git clone https://github.com/olegpolivin/pruningExperiments

In [ ]:
import os
import copy
import time
import pandas as pd
from torchvision.io import read_image
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
from torch import autograd
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision.models import vgg16
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.utils.prune as prune
from heapq import nsmallest
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
import scipy.io
from os.path import join
from os import listdir
from torch.utils import data
from PIL import Image
import torchvision.datasets
from sklearn.model_selection import train_test_split
import psutil
import GPUtil 
from ptflops import get_model_complexity_info
import sys 
sys.path.append(os.path.abspath("/content/pruningExperiments"))
import pruning_loop

Verificamos la grafica

In [ ]:
!nvidia-smi

In [ ]:
BATCH_SIZE = 32

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Importación de dataset

In [ ]:
class Cub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root, train=True, transform=None, loader=default_loader, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.loader = default_loader
        self.train = train

        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows(): 
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1 
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

Transformación de las imágenes

In [ ]:
transform = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Función de entrenamiento 

In [ ]:


def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes,nclas, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_bal_acc = 0.0

    val_bal_acc = []
    val_acc = []
    val_loss = []

    train_bal_acc = []
    train_acc = []
    train_loss = []


    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        start_time_epoch = time.perf_counter()
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            CF = np.zeros((nclas,nclas)) # Confusion matrix

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device,non_blocking=True)
                labels = labels.to(device,non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                for i in range(len(labels.data)):
                    CF[labels.data[i]][preds[i]] +=1

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            recalli = 0
            for i in range(nclas):
                TP = CF[i][i]
                FN = 0
                for j in range(nclas):
                    if i!=j:
                        FN+=CF[i][j]
                if (TP+FN) !=0:
                    recalli+= TP/(TP+FN)
            epoch_bal_acc = recalli/nclas

            if phase == 'val':
                val_bal_acc.append(epoch_bal_acc)
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)
            else:
                train_bal_acc.append(epoch_bal_acc)
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Balanced Acc: {epoch_bal_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_bal_acc > best_bal_acc:
                best_acc = epoch_acc
                best_bal_acc = epoch_bal_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        end_time_epoch = time.perf_counter()
        elapsed_time = end_time_epoch - start_time_epoch
        


        print('Wallclock time for epoch {}: {:.2f} seconds'.format(epoch, elapsed_time))
        print('Current RAM: ',psutil.virtual_memory())
        print('Memory status: ', GPUtil.showUtilization)

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    #print(f'Best val Acc: {best_acc:4f}')
    print(f'Best val Balanced Acc: {best_bal_acc:4f}')

    print('Validation:')
    print('Val_bal_acc:', val_bal_acc)
    print('Val_acc:', val_acc)
    print('Val_loss:', val_loss)

    print('Training:')
    print('Train_bal_acc:', train_bal_acc)
    print('Train_acc:', train_acc)
    print('Train_loss:', train_loss)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Función para el calculo del tiempo de inferencia

In [ ]:
def infer_time(model, dataloader):
    start_time = time.time()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device,non_blocking=True)
            labels = labels.to(device,non_blocking=True)
            outputs = model(inputs)
    end_time = time.time()
    return end_time - start_time

Local L1 unstructured pruning

In [ ]:

def imageNetPruningL1(pruningAmount, epochs, finetuning, transform, dataset):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 

  if pruningAmount != 0:
    features = []
    classifier = []

    for n,p in model.named_parameters():
      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))

    for x in features:
      prune.l1_unstructured(model.features[x], 'weight', amount=pruningAmount)
    for x in classifier:
      prune.l1_unstructured(model.classifier[x], 'weight', amount=pruningAmount)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)

  exp = pruning_loop.PruningExperiment()
  FLOPS, compresion = exp.calculate_prune_metrics(model,ds['val'], device)
  print('FLOPS: ',FLOPS)
  if epochs > 0:
    train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)

  return 

Función para la obtención de dispersión de un modelo

In [ ]:
def get_sparsity(model, pruned):
  pruning = []
  for n_module, module in model.named_modules():
      zeros = 0
      elem = 0
      if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
        if pruned:
          for n_buffer, buffer in module.named_buffers():
            if 'weight_mask' in n_buffer:
              zeros += torch.sum(buffer == 0).item()
              elem += buffer.nelement()
        else:
          for n_param, param in module.named_parameters():
           if 'weight' in n_param:
            zeros += torch.sum(param == 0).item()
            elem += param.nelement()
      if elem!=0:
        print('Módulo: ',n_module,' Porcentaje de pruning: ', zeros/elem)
        pruning.append(zeros/elem)
  print(pruning)
  return zeros/elem
  


Función para la aplicación de pruning global

In [ ]:
def imageNetPruningCUBL1GlobalUnstr(pruningAmount, epochs, finetuning, transform, dataset): 
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 
  parameters_to_prune = [
      (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d or type(m) == torch.nn.Linear, model.modules())
  ]

  prune.global_unstructured(
      parameters_to_prune,
      pruning_method=prune.L1Unstructured,
      amount=pruningAmount
  )

  get_sparsity(model = model, pruned = True)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)
  exp = pruning_loop.PruningExperiment()
  FLOPS, compresion = exp.calculate_prune_metrics(model,ds['val'], device)
  print('FLOPS: ',FLOPS)
  return train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)
  



Local Structured pruning

In [ ]:

def imageNetPruningLnStructured(pruningAmount, epochs, finetuning, transform, dataset, norm):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 

  if pruningAmount != 0:
    features = []
    classifier = []
    for n,p in model.named_parameters():

      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[1] != '6' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))
    for x in features:
      prune.ln_structured(model.features[x], 'weight', amount=pruningAmount, n = norm, dim=1)
      torch.nn.utils.prune.remove(model.features[x], 'weight')
    for x in classifier:
      prune.ln_structured(model.classifier[x], 'weight', amount=pruningAmount, n= norm, dim=1)
      torch.nn.utils.prune.remove(model.classifier[x], 'weight')

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)
  exp = pruning_loop.PruningExperiment()
  FLOPS, compresion = exp.calculate_prune_metrics(model,ds['val'], device)
  print('FLOPS: ',FLOPS)
  if epochs > 0:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)
  
  val_time = infer_time(model, ds['val'])
  print(f"Tiempo de inferencia en el conjunto de validación: {val_time:.2f} segundos")
  return 

Lottery ticket Structured

In [ ]:
def lotteryTicketPruning(pruningAmount, iteration_epochs, iterations, transform, dataset, norm):

  s = 1-pruningAmount #Sparsity = 1 - %pruning

  amnt = s**(1/iterations) # Sparsity each iteration
  sparsity = 1

  init_parameters = {}

  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 


  #Copy original parameters
  for n,p in model.named_parameters():
      init_parameters[n] = p.data.clone()

  for x in range(iterations):
    print('Current sparsity')
    print(sparsity)
    sparsity = sparsity*amnt
    

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    model = model.to(device)
    if iteration_epochs != 0:
      model = train_model(model, criterion, optimizer, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=iteration_epochs)
    exp = pruning_loop.PruningExperiment()
    FLOPS, compresion = exp.calculate_prune_metrics(model,ds['val'], device)
    print('FLOPS: ',FLOPS)

    for i,module in enumerate(model.modules()):
      if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        prune.ln_structured(module, 'weight', amount=1-sparsity, n= norm, dim=1 )


    for n,p in model.named_parameters():
          n2 = n.replace('_orig','')
          if n2 in init_parameters:
            p.data = init_parameters[n2].clone()
    model = model.to(device)
    
    for i,module in enumerate(model.modules()):
      if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        prune.remove(module, 'weight')
        #print('test2')
        #print(module.weight)

  

Regularización sobre el modelo

In [ ]:

def imageNetPruningLnStructuredRegul(pruningAmount, epochs, finetuning, transform, dataset, norm, preepochs):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 


  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001) #Penalizamos en función de L2-norm
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)
  print('Pretraining: ')
  train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=preepochs)

  if pruningAmount != 0:
    features = []
    classifier = []
    for n,p in model.named_parameters():
      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[1] != '6' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))
    for x in features:
      prune.ln_structured(model.features[x], 'weight', amount=pruningAmount, n = norm, dim=1)
    for x in classifier:
      prune.ln_structured(model.classifier[x], 'weight', amount=pruningAmount, n= norm, dim=1)

  

  optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0) #Quitamos la regularización

  return train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)